## Creation of combined RPE-PR and single cell models 
using different CS-models (Lukowski & Voight datasets) and the current Human1 model



In [ ]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [ ]:
# create folder for models
from pathlib import Path
from datetime import datetime

monthyear = datetime.now().strftime("%B%Y")
folder_new_models = Path().cwd() / "models" / 'cs_mods' / 'models_outer_retina' /monthyear
folder_new_models.mkdir(parents=True, exist_ok=True)

In [ ]:
# load RPE, PR, Recon3D, and basis Human1 models from cs mods folder
from cobra.io import read_sbml_model
mod = read_sbml_model(Path().cwd() / "models" / 'template' / 'Human-GEM_28062023.xml' )
mod_RPE = read_sbml_model(Path().cwd() / "models"/  'cs_mods' / 'model_1_RPE__VoigtEtAl2019_.xml')
mod_PRrod = read_sbml_model(Path().cwd() / "models"/  'cs_mods' / 'model_2_RodPhotoreceptors__LiangEtAl_2019_.xml' )
mod_PRcone = read_sbml_model(Path().cwd() / "models"/  'cs_mods' / 'model_3_ConePhotoreceptors__LiangEtAl_2019_.xml' )
mod_PRrod_Lukowski = read_sbml_model(Path().cwd() / "models"/  'cs_mods' / 'model_1_AverageRodPRC0,C1,C2,C3,C4,C7(%).xml' )
mod_PRrod_Lukowski.jname = 'AverageRodPR_Lukowski' 
mod_PRrod_Lukowski.id = 'AverageRodPR_Lukowski' 
mod_PRcone_Lukowski = read_sbml_model(Path().cwd() /  'cs_mods' / 'model_6_C10ConePR(%).xml' )
mod_PRcone_Lukowski.name = 'C10ConePR_Lukowski' 
mod_PRcone_Lukowski.id = 'C10ConePR_Lukowski' 

In [ ]:
# modify context specific models
from src.modify_model import add_all_EX_rxns, remove_compartment

for m in [mod_RPE, mod_PRcone, mod_PRrod, mod_PRrod_Lukowski, mod_PRcone_Lukowski]: # add ATP DM hydrolysis and lost EX rxns to context specific models
    m.add_reactions([mod.reactions.get_by_id('MAR03964').copy()]) # make sure it contains ATP hydrolysis rxn
    m = add_all_EX_rxns(mod,m) # add lost EX rxns
    for r in [r for r in m.reactions if len(r.products)==0]: 
        r.bounds = (0,1000) # close lb for exchange reactions

In [ ]:
# remove nucleus ('n') from photoreceptor models    

# INSTEAD OF REMOVING NUCLEUS KEEP NUCLEUS IN PR (AFTER OPENING UP INNER RETINA SUPPLY)

# mod_PRcontrol = remove_compartment(mod.copy(), 'n')  # remove nucleus ('n') from PR model
# mod_PRcone = remove_compartment(mod_PRcone.copy(), 'n')  # remove nucleus ('n') from PR model
# mod_PRrod = remove_compartment(mod_PRrod.copy(), 'n')  # remove nucleus ('n') from PR model
# mod_PRrod_Lukowski = remove_compartment(mod_PRrod_Lukowski.copy(), 'n')  # remove nucleus ('n') from PR model
# mod_PRcone_Lukowski = remove_compartment(mod_PRcone_Lukowski.copy(), 'n')  # remove nucleus ('n') from PR model

mod_PRcontrol = mod.copy()
mod_PRcone = mod_PRcone.copy()
mod_PRrod = mod_PRrod.copy()
mod_PRrod_Lukowski = mod_PRrod_Lukowski.copy()
mod_PRcone_Lukowski = mod_PRcone_Lukowski.copy()

In [ ]:
    from src.modify_model import  add_id_suffix

    # add suffix to all ids in RPE and PR models

    # Human1
    mod_PRcontrol = add_id_suffix(mod_PRcontrol, '_PR')
    mod_RPEcontrol = add_id_suffix(mod.copy(), '_RPE')  

    # CS models
    mod_PRrod = add_id_suffix(mod_PRrod, '_PR')
    mod_PRcone = add_id_suffix(mod_PRcone, '_PR')
    mod_PRcone_Lukowski = add_id_suffix(mod_PRcone_Lukowski, '_PR')
    mod_PRrod_Lukowski = add_id_suffix(mod_PRrod_Lukowski, '_PR')
    mod_RPE = add_id_suffix(mod_RPE, '_RPE')

In [ ]:
    # add RPE-PR interface reactions and metabolites to RPE and PR models

    from src.combine_RPE_PR import add_interface_RPE, add_interface_PR
    # Human1
    mod_PRcontrol = add_interface_PR(mod_PRcontrol, 'e_PR')
    mod_RPEcontrol = add_interface_RPE(mod_RPEcontrol, 'e_RPE')

    # CS models
    mod_RPE = add_interface_RPE(mod_RPE, 'e_RPE')
    mod_PRrod = add_interface_PR(mod_PRrod, 'e_PR')
    mod_PRcone = add_interface_PR(mod_PRcone, 'e_PR')
    mod_PRrod_Lukowski = add_interface_PR(mod_PRrod_Lukowski, 'e_PR')
    mod_PRcone_Lukowski = add_interface_PR(mod_PRcone_Lukowski, 'e_PR')

In [ ]:
    # fuse RPE and PR models   
    from src.modify_model import fix_compartment_dict, add_rxns2groups
    from src.combine_RPE_PR import fuse_RPE_PR

    # Human1
    mod_RPE_PRcontrol = fuse_RPE_PR(mod_RPEcontrol.copy(),mod_PRcontrol)

    # CS models
    mod_RPE_PRrod = fuse_RPE_PR(mod_RPE.copy(),mod_PRrod)
    mod_RPE_PRcone = fuse_RPE_PR(mod_RPE.copy(),mod_PRcone)
    mod_RPE_PRrod_Lukowski = fuse_RPE_PR(mod_RPE.copy(),mod_PRrod_Lukowski)
    mod_RPE_PRcone_Lukowski = fuse_RPE_PR(mod_RPE.copy(),mod_PRcone_Lukowski)

In [ ]:
from src.combine_RPE_PR import del_RPE_PR_dupl_rxns

# delete duplicate reactions in RPE-PR models

mod_RPE_PRcontrol = fix_compartment_dict(mod_RPE_PRcontrol)

mod_RPE_PRrod = fix_compartment_dict(mod_RPE_PRrod)
mod_RPE_PRrod = add_rxns2groups(mod_RPE_PRrod)
mod_RPE_PRrod = del_RPE_PR_dupl_rxns(mod_RPE_PRrod)

mod_RPE_PRcone = fix_compartment_dict(mod_RPE_PRcone)
mod_RPE_PRcone = add_rxns2groups(mod_RPE_PRcone)    
mod_RPE_PRcone = del_RPE_PR_dupl_rxns(mod_RPE_PRcone)

mod_RPE_PRrod_Lukowski = fix_compartment_dict(mod_RPE_PRrod_Lukowski)
mod_RPE_PRrod_Lukowski = add_rxns2groups(mod_RPE_PRrod_Lukowski)
mod_RPE_PRrod_Lukowski = del_RPE_PR_dupl_rxns(mod_RPE_PRrod_Lukowski)

mod_RPE_PRcone_Lukowski = fix_compartment_dict(mod_RPE_PRcone_Lukowski)
mod_RPE_PRcone_Lukowski = add_rxns2groups(mod_RPE_PRcone_Lukowski)
mod_RPE_PRcone_Lukowski = del_RPE_PR_dupl_rxns(mod_RPE_PRcone_Lukowski)

In [ ]:
# remove PR exchange reactions from RPE-PR models (KEEP PR EXCHANGE REACTIONS IN FOR SINGLE MODELS)
# for m in [mod_RPE_PRrod, mod_RPE_PRcone, mod_RPE_PRrod_Lukowski, mod_RPE_PRcone_Lukowski, mod_RPE_PRcontrol]:
#    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
#    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    
for m in [mod_RPE_PRrod, mod_RPE_PRcone, mod_RPE_PRrod_Lukowski, mod_RPE_PRcone_Lukowski, mod_RPE_PRcontrol]:  
    for r in [r for r in m.reactions if len(r.products) == 0 ]:           # close all exchange reactions
        r.bounds=(0,0)
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) 

In [ ]:
from cobra.io import write_sbml_model

write_sbml_model(mod_RPE_PRcontrol, folder_new_models / 'mod_RPE_PRcontrol.xml')
write_sbml_model(mod_RPE_PRrod, folder_new_models / 'mod_RPE_PRrod_Liang.xml')
write_sbml_model(mod_RPE_PRcone, folder_new_models / 'mod_RPE_PRcone_Liang.xml')
write_sbml_model(mod_RPE_PRrod_Lukowski, folder_new_models / 'mod_RPE_PRrod_Lukowski.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski, folder_new_models / 'mod_RPE_PRcone_Lukowski.xml')

In [ ]:
def create_single_model(model, to_keep):
    """
    Create single model from RPE-PR model

    Parameters
    ----------
    m : cobra.Model
        RPE-PR model
    to_keep : str
        cell to keep in RPE-PR model
    
    Returns
    -------
    cobra.Model
        RPE model
    
    """
    m = model.copy()
    reactions_to_remove = []
    compartments = [c for c in list(mod_RPE_PRcone_Lukowski.compartments) if to_keep in c]
    for r in m.reactions:
        # list of compartments in reaction that are not in the RPE compartment list
        l = [x for x in list(r.compartments) if x not in compartments]
        # remove reaction if it contains a compartment that is not in the RPE compartment list
        if len(l) > 0:
            reactions_to_remove.append(r.id)

    m.remove_reactions(reactions_to_remove)
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    return m   

In [ ]:
# create single cell models from RPE-PR models
mod_RPE_PRcontrol_singleRPE = create_single_model(mod_RPE_PRcontrol, 'RPE')  
mod_RPE_PRcontrol_singlePR = create_single_model(mod_RPE_PRcontrol, 'PR')

mod_RPE_PRrod_singleRPE = create_single_model(mod_RPE_PRrod, 'RPE')
mod_RPE_PRcone_singleRPE = create_single_model(mod_RPE_PRcone, 'RPE')
mod_RPE_PRrod_singlePR = create_single_model(mod_RPE_PRrod, 'PR')
mod_RPE_PRcone_singlePR = create_single_model(mod_RPE_PRcone, 'PR')

mod_RPE_PRrod_Lukowski_singleRPE = create_single_model(mod_RPE_PRrod_Lukowski, 'RPE')
mod_RPE_PRcone_Lukowski_singleRPE = create_single_model(mod_RPE_PRcone_Lukowski, 'RPE')
mod_RPE_PRrod_Lukowski_singlePR = create_single_model(mod_RPE_PRrod_Lukowski, 'PR')
mod_RPE_PRcone_Lukowski_singlePR = create_single_model(mod_RPE_PRcone_Lukowski, 'PR')

In [ ]:
#  save single RPE-PR models
write_sbml_model(mod_RPE_PRrod_singleRPE, folder_new_models / 'mod_RPE_PRrod_singleRPE_Liang.xml')
write_sbml_model(mod_RPE_PRcone_singleRPE, folder_new_models / 'mod_RPE_PRcone_singleRPE_Liang.xml')
write_sbml_model(mod_RPE_PRrod_singlePR, folder_new_models / 'mod_RPE_PRrod_singlePR_Liang.xml')
write_sbml_model(mod_RPE_PRcone_singlePR, folder_new_models / 'mod_RPE_PRcone_singlePR_Liang.xml')

write_sbml_model(mod_RPE_PRrod_Lukowski_singleRPE, folder_new_models / 'mod_RPE_PRrod_Lukowski_singleRPE.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski_singleRPE, folder_new_models / 'mod_RPE_PRcone_Lukowski_singleRPE.xml')
write_sbml_model(mod_RPE_PRrod_Lukowski_singlePR, folder_new_models / 'mod_RPE_PRrod_Lukowski_singlePR.xml')
write_sbml_model(mod_RPE_PRcone_Lukowski_singlePR, folder_new_models / 'mod_RPE_PRcone_Lukowski_singlePR.xml')